In [ ]:
%config Completer.use_jedi = False

We import the different things we need, then we pre-process the data such as in the training notebook
===

In [ ]:
from pathlib import Path
import importlib.util

import logging
logging.getLogger().setLevel(logging.INFO)

from sklearn import tree
from sklearn.base import BaseEstimator
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score, balanced_accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn_pandas import DataFrameMapper

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

import pandas as pd

import predictsignauxfaibles.models
from predictsignauxfaibles.data import SFDataset
from predictsignauxfaibles.config import OUTPUT_FOLDER, IGNORE_NA
from predictsignauxfaibles.pipelines import run_pipeline
from predictsignauxfaibles.utils import load_conf

In [ ]:
conf = load_conf("default")

In [ ]:
train = conf.TRAIN_DATASET
train.sample_size = 1e4

test = conf.TEST_DATASET
test.sample_size = 1e4

In [ ]:
savepath = None # change it to be a filepath if you wish to save train and test data locally

train.fetch_data().raise_if_empty()
test.fetch_data().raise_if_empty()
logging.info("Succesfully loaded Features data from MongoDB")

if savepath is not None:
    train.data.to_csv(f"{savepath}_train.csv")
    test.data.to_csv(f"{savepath}_test.csv")
    logging.info(f"Saved Features extract to {savepath}")

In [ ]:
train_siren_set = train.data["siren"].unique().tolist()
test.remove_siren(train_siren_set)

In [ ]:
train.replace_missing_data().remove_na(ignore=IGNORE_NA)
train.data = run_pipeline(train.data, conf.TRANSFO_PIPELINE)

test.replace_missing_data().remove_na(ignore=IGNORE_NA)
test.data = run_pipeline(test.data, conf.TRANSFO_PIPELINE)

In [ ]:
train.data

In [ ]:
train = train.data

In [ ]:
test = test.data

In [ ]:
train.loc[train['outcome']==False].isnull().sum()

In [ ]:
train.drop(['paydex_group','siret','periode','outcome','siren','code_naf','time_til_outcome'], axis=1)

We first define and apply a LR, then measure its different metrics
===

In [ ]:
logreg = LogisticRegression()

In [ ]:
logreg.fit(train.drop(['paydex_group','siret','periode','outcome','siren','code_naf','time_til_outcome'], axis=1),train['outcome'])

In [ ]:
test['outcome'] = logreg.predict(test.drop(['paydex_group','siret','periode','outcome','siren','code_naf','time_til_outcome'], axis=1))

In [ ]:
y_pred_lr = logreg.predict(train.drop(['paydex_group','siret','periode','outcome','siren','code_naf','time_til_outcome'], axis=1))

In [ ]:
confusion_matrix(list(train['outcome']),y_pred_lr)

In [ ]:
balanced_accuracy_score(list(train['outcome']),y_pred_lr)

In [ ]:
fbeta_score(list(train['outcome']),y_pred_lr,beta=2)

Then we define and apply a DT of maximum depth equal to 5, then measure its metrics
====

In [ ]:
dectree = tree.DecisionTreeClassifier(max_depth=5)

In [ ]:
dectree.fit(train.drop(['paydex_group','siret','periode','outcome','siren','code_naf','time_til_outcome'], axis=1),train['outcome'])

In [ ]:
tree.plot_tree(dectree) 

In [ ]:
Y = dectree.predict(test.drop(['paydex_group','siret','periode','outcome','siren','code_naf','time_til_outcome'], axis=1))

In [ ]:
decision_tree = tree.DecisionTreeClassifier(random_state=0, max_depth=5)
decision_tree = decision_tree.fit(train.drop(['paydex_group','siret','periode','outcome','siren','code_naf','time_til_outcome'], axis=1), train['outcome'])
r = tree.export_text(decision_tree, feature_names=list(train.drop(['paydex_group','siret','periode','outcome','siren','code_naf','time_til_outcome'], axis=1).columns))
print(r)

In [ ]:
y_pred_dt = dectree.predict(train.drop(['paydex_group','siret','periode','outcome','siren','code_naf','time_til_outcome'], axis=1))

In [ ]:
confusion_matrix(list(train['outcome']),y_pred_dt)

In [ ]:
balanced_accuracy_score(list(train['outcome']),y_pred_dt)

In [ ]:
fbeta_score(list(train['outcome']),y_pred_dt,beta=2)

In [ ]:
precision_recall_fscore_support(list(train['outcome']),y_pred_dt,beta=2,average='binary')

Then we define and apply a DT of maximum depth equal to 5, then measure its metrics
====

In [ ]:
SVM = make_pipeline(StandardScaler(), SVC())

In [ ]:
SVM.fit(train.drop(['paydex_group','siret','periode','outcome','siren','code_naf','time_til_outcome'], axis=1),train['outcome'])

In [ ]:
y_pred_svm = SVM.predict(train.drop(['paydex_group','siret','periode','outcome','siren','code_naf','time_til_outcome'], axis=1))

In [ ]:
confusion_matrix(list(train['outcome']),y_pred_svm)

In [ ]:
balanced_accuracy_score(list(train['outcome']),y_pred_svm)

In [ ]:
precision_recall_fscore_support(list(train['outcome']),y_pred_svm,beta=2,average='binary')